# Assignment 2


You must submit your notebook by running `python3 -m autograder.cli.submission.submit Assignment2.ipynb` from your local repository.

To write legible answers you will need to be familiar with both [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet) and [Latex](https://www.latex-tutorial.com/tutorials/amsmath/)

Before you turn this problem in, make sure everything runs as expected. To do so, restart the kernel and run all cells (in the menubar, select Runtime→→Restart and run all).

#### Show your work!
Whenever you are asked to find the solution to a problem, be sure to also **show how you arrived** at your answer.

Make sure you fill in any place that says "YOUR CODE HERE" or "YOUR ANSWERS HERE", as well as your name below:

## Some Helper Functions

Implement an evaluation function that takes in some board position and player colour and returns a score. 

In [6]:
import chess
import random
from math import inf
from IPython.display import display, clear_output

In [7]:
def evaluation(b: chess.Board, player: bool):
    """
    This function evaluates a board position and returns a score value.
    
    Parameters:
    - board: the chess board that the knight is moving upon
    - player: the colour of the active player (True -> white, False -> black)
    
    Returns:
    - an integer score for a board state
    """
    # Define weights for each factor in the linear function
    WEIGHT_MATERIAL = 1.0
    WEIGHT_MOBILITY = 0.1
    WEIGHT_KING_SAFETY = 0.3
    WEIGHT_TAPERED = 0.2
    WEIGHT_TRAPPED = 0.4  # New weight for trapped pieces

    # Compute the individual evaluation components
    material_score = evaluate_material(board)
    mobility_score = evaluate_mobility(board)
    king_safety_score = evaluate_king_safety(board)
    tapered_score = evaluate_tapered(board)
    trapped_score = evaluate_trapped_pieces(board)  # New trapped evaluation

    # Combine scores using a linear weighted function
    total_score = (WEIGHT_MATERIAL * material_score) +
                  (WEIGHT_MOBILITY * mobility_score) +
                  (WEIGHT_KING_SAFETY * king_safety_score) +
                  (WEIGHT_TAPERED * tapered_score) +
                  (WEIGHT_TRAPPED * trapped_score)

    return total_score

# Function to evaluate if a piece is trapped
def evaluate_trapped_pieces(board):
    trapped_score = 0

    for piece in board.pieces:
        if piece_belongs_to_white(piece):
            trapped_score += evaluate_piece_trap_status(piece, WHITE)
        else:
            trapped_score -= evaluate_piece_trap_status(piece, BLACK)

    white_pieces = board.pieces(chess.WHITE)
    black_pieces = board.pieces(chess.BLACK)

    return trapped_score

# Function to evaluate the trap status of a single piece
def evaluate_piece_trap_status(piece, player):
    trap_penalty = 0

    # # Get all legal moves for the piece
    # legal_moves = get_legal_moves(piece, board)

    # # If the piece has few or no legal moves, consider it trapped
    # IF len(legal_moves) == 0:
    #     trap_penalty += 2  # High penalty for immobility (completely trapped)
    # ELSE IF len(legal_moves) < 2:
    #     trap_penalty += 1  # Lower penalty for restricted mobility

    # # Check if the piece is under attack and has no defenders
    # IF piece_is_under_attack(piece, board) AND piece_has_no_defenders(piece, board):
    #     trap_penalty += 3  # High penalty for vulnerable, undefended pieces

    return trap_penalty

# Function to check if a piece is under attack
def piece_is_under_attack(piece, board):
    attacker_list = get_attackers(piece, board)
    return len(attacker_list) > 0

# Function to check if a piece has defenders
def piece_has_no_defenders(piece, board):
    defender_list = get_defenders(piece, board)
    return len(defender_list) == 0


SyntaxError: invalid syntax (3594564977.py, line 27)

## Q1

Implement the minimax algorithm to choose the best chess move for a given board position, player colour, and look-ahead depth. Make use of your `evaluation()` function here to calculate the score of any individual board state.

In [ ]:
def get_minimax_move(b: chess.Board, player: bool, depth: int):   
    """
    This function chooses the best move for the given board position, player, and depth.
    
    Parameters:
    - board: the chess board that the knight is moving upon
    - player: the colour of the active player (True -> white, False -> black)
    - depth: the number of moves that the algorithmn should look ahead.
    
    Returns:
    A single chess.Move type object.
    """
    # limit to a depth of 6-8 to not fry computer

    def helper(local_b, local_player, local_depth, alpha, beta):
        # YOUR CODE HERE
        if local_depth == 0 or local_b.outcome() > 0:
            return evaluation(local_b, local_player)
        if(local_player == chess.WHITE):
            v = float('-inf')
            for move in local_b.legal_moves:
                new_board = local_b.copy()
                new_board.push(move)
                v = max(v, helper(new_board, !local_player, local_depth-1, alpha, beta))
                if alpha >= beta:
                    break
            return v
        else:
            v = float('inf')
            for move in local_b.legal_moves:
                new_board = local_b.copy()
                new_board.push(move)
                v = min(v, helper(new_board, !local_player, local_depth-1, alpha, beta))
                if alpha >= beta:
                    break
            return v

    # YOUR CODE HERE
    return helper(b, player, depth, float('-inf'), float('inf'))

## Q2

Implement the expectimax algorithm to choose the best chess move for a given board position, player colour, and look-ahead depth. Make use of your `evaluation()` function here to calculate the score of any individual board state.

In [ ]:
def get_expectimax_move(b: chess.Board, player: bool, depth: int):   
    """
    This function chooses the best move for the given board position, player, and depth.
    
    Parameters:
    - board: the chess board that the knight is moving upon
    - player: the colour of the active player (True -> white, False -> black)
    - depth: the number of moves that the algorithmn should look ahead.
    
    Returns:
    A single chess.Move type object.
    """


    # function expectiminimax(node, depth)
    # if node is a terminal node or depth = 0
    #     return the heuristic value of node
    # if the adversary is to play at node
    #     // Return value of minimum-valued child node
    #     let α := +∞
    #     foreach child of node
    #         α := min(α, expectiminimax(child, depth-1))
    # else if we are to play at node
    #     // Return value of maximum-valued child node
    #     let α := -∞
    #     foreach child of node
    #         α := max(α, expectiminimax(child, depth-1))
    # else if random event at node
    #     // Return weighted average of all child nodes' values
    #     let α := 0
    #     foreach child of node
    #         α := α + (Probability[child] × expectiminimax(child, depth-1))
    # return α
    def helper(local_b, local_player, local_depth, alpha, beta):
        # YOUR CODE HERE
        if local_depth == 0 or local_b.outcome > 0:
            return evaluation(local_b, player)
        
        if player == chess.WHITE:
            let

    # YOUR CODE HERE
    raise NotImplementedError

## Q3

Plays against the strongest grading agent, so copy your best agent here.

In [ ]:
def get_best_move(b: chess.Board, player: bool, depth: int):   
    """
    This function chooses the best move for the given board position, player, and depth.
    
    Parameters:
    - board: the chess board that the knight is moving upon
    - player: the colour of the active player (True -> white, False -> black)
    - depth: the number of moves that the algorithmn should look ahead.
    
    Returns:
    A single chess.Move type object.
    """    
    
    # YOUR CODE HERE
    return get_minimax_move(b, player, depth)

## Local Testing
For your convenience, here are some helper functions to run games and allow you to test your agents.

In [ ]:
def get_random_move(b:chess.Board, *_):
    rand = random.Random(0)
    return rand.choice(list(b.legal_moves))

In [ ]:
MAX_DEPTH = 3

def test_game(p1, p2):
    """
    A function that plays a chess game with visuals
    
    Parameters:
    - p1: the first chess agent, playing as the white player
    - p2: the second chess agent, playing as the black player
    
    Returns:
    Nothing, just plays the game one move at a time.
    Press enter in the popup box to play the next move.
    Type 'q' into the popup box and press enter to stop the game (you can also interrupt the kernel if this fails.)
    """
    board = chess.Board()
    current_player = p1

    while not board.is_checkmate()  and not board.is_stalemate() and not board.can_claim_draw():
        clear_output(True)
        
        move = current_player(board, board.turn, MAX_DEPTH)
        board.push(move)
        display(board)

        current_player = p1 if board.turn else p2

        if input() == "q":
            break
    

In [ ]:
# Random Vs. Random
random_agent = get_random_move
test_game(random_agent, random_agent)

In [ ]:
# Minimax Vs. Random
random_agent = get_random_move
minimax_agent = get_minimax_move
test_game(minimax_agent, random_agent)

In [ ]:
# Expectimax Vs. Random
random_agent = get_random_move
expectimax_agent = get_expectimax_move
test_game(expectimax_agent, random_agent)

In [ ]:
# Run several games at once without graphics
def grade_game(p1, p2):
    """
    Grades a single game at a time. 
    (Note: differs from actual grading script, which has some ways to resolve draws.)

    Return legend:
    0: error during game
    1: p1 wins through checkmate
    2: p2 wins through checkmate
    3: draw
    """
    board = chess.Board()
    current_player = p1

    while not board.is_checkmate() and not board.is_stalemate() and not board.can_claim_draw():
        move = current_player(board, MAX_DEPTH, board.turn)
        try:
            board.push(move)
        except:            
            print(f"Error while grading game, move = {move}, current_player = {current_player}")
            print(board)
            return -1

        current_player = p1 if board.turn else p2
    
    outcome = board.outcome()
    if outcome is not None:
        if outcome.winner:
            return 1
        else:
            return 2
    
    return 3

In [ ]:
# Play many games in a row without visuals (faster)
random_agent = get_random_move
best_agent = get_best_move
num_games = 5

print(f"Playing {num_games} games...")
for i in range(1, num_games+1):
    result = grade_game(best_agent, random_agent)

    if result == 0:
        print(f"Game {i}: error pushing a move during gameplay")
        
    elif result == 1:
        print(f"Game {i}: p1 wins")
        
    elif result == 2:
        print(f"Game {i}: p1 loses")
    
    elif result == 3:
        print(f"Game {i}: draw")